In [ ]:
# -- besure to put the title, name, email and date here in a markdown cell! 

In [1]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))
import warnings
warnings.filterwarnings('ignore')
# ------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# -- sklearn modules 
from sklearn.model_selection import train_test_split   #- partition train/test split 
from sklearn.tree import DecisionTreeClassifier        #- create a decison tree 
from sklearn.ensemble import ExtraTreesClassifier      #- random trees 
from sklearn.ensemble import RandomForestClassifier    #- random forest classifier 

# -- we need this to make our pipeline 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# -- categorical encoders -- 
from category_encoders import *


# -- need this to render charts in notebook -- 
%matplotlib inline


## Project 4,  Who is Likely to Default?

The bosses were excited about your promotion model, but now they have a whole lot of new loans comming through. Can you identify which of these new loans are likely to default? You’ll build a model that can be used to hedge the institution's default risk. To do this you will predict which loans will end up in default status (0 – Paid in Full) or (1 – Default) status. In this project you will draw on your Python ML skills to analyze loan default risk. You are given two datasets one training dataset and one prediction dataset. 

The following Tasks have been dived into Three(3) parts, simply look at the section's **Todos** for your project's required tasks. If there is a question, simply add a markdown cell and answer the question. As always feel free to add additional cells and analysis as you dig into the data.  


### Part 1
1. Stage data
2. Clean up column names 
3. Describe data 
4. Transform & create new columns 
5. Explore likely predictors  

### Part 2.

6. Partition into 75/25 split 

7. Train ExtraTrees, RandomForest and GradientBoosting Classifier
    - onehot encoding 
    - target encoding  
    - Tune Parameters 
    - feature importance 
    
8. Evaluate & Compare Performance of Each 
    - confusion matrix 
    - calculate accuracy 
    - Receiver Operator Characteristic (ROC Chart) 
    - Area Under the Curve (AUC)
    - Recal(TPR) @4% FPR  


### Part 3.  

10. Write up your thoughts.
    - which model perfomed the best? 
    - Did the models have similar feature importance? 
    - For your best performing RandomForest and GBM models at a false positive rate of 4% what is the true positive rate? and score thresholds. What would be the business rule you would tell the business to implement? 
    - Given the loan amount, how much would your model save in identified defaults? how did you calculate this? 
    
11. Make a new prediction file, submit to kaggle 
    - apply your best model to the predict dataset and submit your prediction to kaggle. 
  

# Part 1. 
## 1. Stage 
----- 
Import both loans.csv to df,  and the predict.csv to predict 



<div class="alert alert-info"> 💡 <strong> TODO </strong>
 
1. Read loan.csv into a dataframe named df 
2. Read predict.csv into a dataframe named predict 
3. use df.head() to display the first 5 records of each 

</div>

```python 
df = pd.read_csv("../Week_3/Data/bank-additional-full.csv", header=0, sep=';', quotechar='"')
df.head()
```

In [2]:
df = pd.read_csv("./data/loans.csv")
df.head()

,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens,ID,loan_default
0,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0,20,0
1,215952.0,Short Term,739.0,1454735.0,< 1 year,Rent,Debt Consolidation,39277.75,13.9,NaN,20.0,0.0,669560.0,1021460.0,0.0,0.0,100,0
2,99999999.0,Short Term,728.0,714628.0,3 years,Rent,Debt Consolidation,11851.06,16.0,76.0,16.0,0.0,203965.0,289784.0,0.0,0.0,110,0
3,99999999.0,Short Term,740.0,776188.0,< 1 year,Own Home,Debt Consolidation,11578.22,8.5,25.0,6.0,0.0,134083.0,220220.0,0.0,0.0,130,0
4,99999999.0,Short Term,743.0,1560907.0,4 years,Rent,Debt Consolidation,17560.37,13.3,NaN,10.0,1.0,225549.0,496474.0,1.0,0.0,140,0


In [4]:
df['Credit_Score'] = np.where((df["Credit_Score"] > 1000),df['Credit_Score']/10,df['Credit_Score'])
df['Credit_Score'].max()

751.0

In [12]:
df["current_job_years"] = df["Years_in_current_job"].map({'10+ years': 10,
                                                          '9 years': 9,
                                                          '8 years': 8 })
df["current_job_years"].value_counts()

10.0    21710
8.0      3242
9.0      2750
Name: current_job_years, dtype: int64

In [6]:
df["current_job_years"]  = np.where(df["Years_in_current_job"] == "10+ years", 10, df["Years_in_current_job"] )
#df["current_job_years"]  = np.where(df["Years_in_current_job"] == "10+ years", 10, 0 )
df["current_job_years"].value_counts()

10          21710
2 years      6451
< 1 year     5742
3 years      5688
5 years      4802
1 year       4564
4 years      4308
6 years      3974
7 years      3942
8 years      3242
9 years      2750
Name: current_job_years, dtype: int64

## 2.  Clean up Column Names

*It's just not fun dealing with ill-formed columns*

<div class="alert alert-info"> 💡 <strong> TODO </strong>
 
- remove leading and trailing characters
- replace spaces with underscores _ 
- replace "." with underscores _ 
- change case to lower case
- remove various special characters

</div>



## 3. Describe data


<div class="alert alert-info"> 💡 <strong> TODO </strong>
 
- check target, counts and PCTs 
- generate descriptive statisicts with describe 
- check for nulls 

</div>


## 4. Transform & Clean Variables 

- creditscore  

- current loan amount


<div class="alert alert-info"> 💡 <strong> TODO </strong>
    
<b> transform years_in_current_job </b>


- create a new variable current_job_years by doing the following

    - if 10+ years then current_job_years = 10
    - if 1-9 years then parse into appropriate years 
    - if < 1 year then 0 
    - be sure do do a value_counts() on the new variable, afterwards to show that you've transformed it corectly. 
 
<b> fix creditscore </b> 

- if creditscore > 1000 then divide by 10 

<b> fix current_loan_amount </b>

- if current_loan_amount == 99999999, then set current_loan_amt to -1 


        

</div>

## 5. Explore likely predictors 

<div class="alert alert-info"> 💡 <strong> TODO </strong>
 
- Pick 3 character columns 
    - crosstab 
    - frequency count by target  
    - percentage by target 

- Pick new  variable "current_job_years" treat it as a character and perform the following   
    - crosstab 
    - frequency count by target  
    - percentage by target 

- Pick credit_score numeric columns 
    - cut them into bins 
    - frequency count by target 
    - percentage by target 
    
- Pick annual_income numeric column 
    - cut them into bins 
    - frequency count by target 
    - percentage by target 
    
- Pick two other numeric columns and explore them 
    - cut them into bins 
    - frequency count by target 
    - percentage by target 
    
    
- And you of course can do any other exploration you think will be useful. 

- use seaborn's pairs plot like we've seen a couple times already. 

**Answer**
1. does anything stand out to you about these plots? 
2. do you think any of these will be good predictors of our target? 
3. can you think of anther graphic which might be useful to identify default vs. non-default. 

</div>

# Part 2.
## 6. Partition into 75/25 split 

<div class="alert alert-info"> 💡 <strong> TODO </strong>
    
- partition into 75/25 split.. 
- print out the percentages 

</div>


## 7. Train an ExtraTreesClassifier, a RandomForestClassifier and a GBM 

-----

<div class="alert alert-info"> 💡 <strong> TODO </strong>
    
- Deterimine which columns go into 
    - target
    - numeric features
    - categorical features 
    
- Create ExtraTree pipeline
    - categorical handling - up to you to choose how you'd want to handle 
        - missing value handling, categorical encoding 
        - use a loop or gridsearch to evaluate hyperparameters: 
            - n_estimators = 100, 150, 200 
            - max_depth = 5, 10, 20
        - evaluate on TEST set, select the ExtraTree model hyperparameters with best AUC 
        - print Train/Test AUC & Accuracy
        - plot Variable Importance 
        
- Create RandomForest pipeline
    - categorical handling - up to you to choose how you'd want to handle 
        - missing value handling, categorical encoding 
        - use a loop or gridsearch  to evaluate hyperparameters: 
            - n_estimators = 100, 150, 200 
            - max_depth = 5, 10, 20
        - evaluate on TEST set, select the RandomForest model hyperparameters with best AUC 
        - print best hyperparameter combination 
        - print Train/Test AUC & Accuracy
        - plot Variable Importance 
         
        
        
- Create GradientBoostingMaching(GBM) pipeline
    - categorical handling - up to you to choose how you'd want to handle 
        - missing value handling, categorical encoding 
        - use a loop or gridsearch to evaluate hyperparameters: 
            - learning_rate = 0.01, 0.05, 0.1, 0.5, 1
            - n_estimators = 100, 200 
        - evaluate on TEST set, select the GradientBoostingMaching model hyperparameters with best AUC 
        - print best hyperparameter combination 
        - print Train/Test AUC & Accuracy
        - plot Variable Importance 

> hint make your life easier and make functions! 
</div>


## 8. Evaluate 

<div class="alert alert-info"> 💡 <strong> TODO </strong>
    
FOR EACH OF YOUR BEST classifiers calculate the following 
1. calculate the varaible importance
2. confusion matrix of test counts 
3. confusion matrix of test percentages 
4. accuracy of both train and test vs baseline (i.e. the do nothing case) 
5. create a ROC chart with AUC calcualtion comparing training and testing 
6. make a table of FPR, TPR and Threshold from 0 - 10% FPR 


> hint make your life easier and make functions! 
</div>





## 9. Write up.

<div class="alert alert-info"> 💡 <strong> TODO </strong>

- which model perfomed the best? What was the best hyperparamert combination?  
- Did the models have similar feature importance? did different categorical encodings make a difference? and which do you think is more likely to lead to overfitting and why? 
- for your best performing ExtraTree, RandomForest and GBM models at a false positive rate of 4% what is the true positive rate? and score thresholds. What would be the business rule you would tell the business to implement? 

- If you apply gthe business rule at 4% FPR on your best model,  how many True Positives would your model find on the Test Set vs False Positives, if you made \\$1000 and -\\$200 for each false positive how much would your model make? 



</div>


